In [294]:
import pandas as pd
import json
import requests


In [295]:
import argparse
from pandas.io.json import json_normalize

In [487]:
class ImmobiliareApi:
    '''
    page_number : number of the page, max 80, 25 advert for each (2000 total)
    minimum_price - maximum_price : the range of the price
    province : RM
    contract_type : 
        - 1: Vendita,
        - 2: Affitto, 
        - 14: Asta 
    advert_type:
        - 1: Case - Appartamenti
        - 6: Nuove costruzioni
        - 4: Terreni
    '''

    def __init__(self, 
                minimum_price,
                maximum_price,
                contract_type=1 ,
                advert_type=1,
                province='RM'):
        
        # self.page_number = str(page_number)
        self.minimum_price = str(int(minimum_price))
        self.maximum_price = str(int(maximum_price))
        self.province = str(province)
        self.contract_type = str(int(contract_type))
        self.advert_type = str(advert_type)
        
    def single_page_call(self, page_number):
        
        assert (int(page_number)>0) & (int(page_number)<81)

        page_number = str(page_number)
        
        url = f"https://www.immobiliare.it/api-next/search-list/real-estates/?path= &idContratto={self.contract_type}&criterio=rilevanza&pag={page_number}&prezzoMinimo={self.minimum_price}&prezzoMassimo={self.maximum_price}&idProvincia={self.province}&idCategoria={self.advert_type}"
        
        response = requests.request("GET", url, headers={}, data={})
        
        data = json.loads(response.text)

        index_name = '_'.join([
                self.province, \
                self.minimum_price, \
                self.maximum_price, \
                page_number, \
                self.contract_type, \
                self.advert_type, \
                ])

        data['index_name'] = index_name
        
        return data

    def call_report(self):
        
        tmp_call = self.single_page_call(page_number=1)
        # print( tmp_call)
        print(
            '\n# API Call Report', \
            '\n\n## Parameters', \
            '\nprovince :' , self.province, \
            '\nminimum_price :' , self.minimum_price, \
            '\nmaximum_price :' , self.maximum_price, \
            '\ncontract_type :' , self.contract_type, \
            '\nadvert_type : ', self.advert_type, \
            '\n\n## Statistics', \
            '\nTotal ads :', tmp_call['totalAds'], \
            '\nMax pages :', tmp_call['maxPages'], \
        )
        

        

In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Ingest CSV data to Postgres')

    parser.add_argument('--user', required=True, help='user name for postgres')
    parser.add_argument('--password', required=True, help='password for postgres')
    parser.add_argument('--host', required=True, help='host for postgres')
    parser.add_argument('--port', required=True, help='port for postgres')
    parser.add_argument('--db', required=True, help='database name for postgres')
    parser.add_argument('--table_name', required=True, help='name of the table where we will write the results to')
    parser.add_argument('--url', required=True, help='url of the csv file')

    args = parser.parse_args()

    main(args)

In [488]:
api_call = ImmobiliareApi(minimum_price=1e5, maximum_price=2e5)

In [489]:
# api_call.single_page_call(page_number=80)


In [490]:
api_call.call_report()


# API Call Report 

## Parameters 
province : RM 
minimum_price : 100000 
maximum_price : 200000 
contract_type : 1 
advert_type :  1 

## Statistics 
Total ads : 18127 
Max pages : 726


In [493]:
# pd.json_normalize(api_data, \
#                 record_path=['results'], \
#                 record_prefix='results_', \
#                 meta=['index_name','currentPage'], \
#                 meta_prefix='meta_', \
#                 errors='ignore', \
#                 sep='.'
#                 )


In [145]:
# import gzip
# def json_zip_writer(j, target_key):
#     f = gzip.open(target_key, 'wb')
#     f.write(json.dumps(j).encode('utf-8'))
#     f.close()


In [146]:
#  json_zip_writer(parsed['seoData'], './example_json.gz')

In [147]:
# pd.read_json('./example_json.gz').iloc[24]['realEstate']
# pd.read_json('./example_json.gz')

In [148]:
all_advert_data = []

for item_no in range(25):

    advert = data['results'][item_no]['realEstate']

    advert_data = {}
    

    for key_i,value_i in advert.items():

        if isinstance(value_i,list):
            advert_data[key_i] = value_i
            
        elif isinstance(value_i,dict):
            pass

        else:
            advert_data[key_i] = value_i


    advert_properties = advert['properties'][0]
    for key_i,value_i in advert_properties.items():

        if isinstance(value_i,list):
            advert_data[key_i] = value_i
            
        elif isinstance(value_i,dict):
            for key_j,value_j in value_i.items():
                colname_j = f"{key_i}_{key_j}"
                advert_data[colname_j] = value_j

        else:
            advert_data[key_i] = value_i
    
    all_advert_data.append(advert_data) 
    # break


In [149]:
# for k in advert.keys():
#     if ('properties' not in k) or ('agency' not in k):
#         print(k)
#         print(advert[k] )

In [150]:
pd.DataFrame(all_advert_data)

,dataType,contract,id,isNew,luxury,properties,propertiesCount,title,type,visibility,...,location_region,location_province,location_macrozone,location_microzone,location_city,location_nation,hasElevators,typologyGA4Translation,ga4features,ga4Garage
0,list-real-estate,sale,99848032,False,False,"[{'caption': 'VIA DI DRAGONCELLO, NUOVA COSTRU...",19,Appartamenti e Attici di nuova costruzione a Roma,ad,supervetrina,...,Lazio,Roma,"Acilia, Casal Bernocchi, Centro Giano, Dragona...",Acilia,Roma,"{'id': 'IT', 'name': 'Italia', 'keyurl': 'Ital...",False,Progetto,"[Esposizione interna, Esposizione esterna, Acc...",144 in parcheggio/garage comune
1,list-real-estate,sale,99702010,False,False,[{'caption': 'OASI Smart Living - Nuovo esclus...,9,Appartamenti di nuova costruzione a Fiumicino,ad,supervetrina,...,Lazio,Roma,"Fiumicino Paese, Focene, Isola Sacra, Parco Le...",Fiumicino Paese,Fiumicino,"{'id': 'IT', 'name': 'Italia', 'keyurl': 'Ital...",False,Progetto,"[Esposizione interna, Esposizione esterna, Acc...",NaN
2,list-real-estate,sale,100715187,False,False,[{'caption': 'NUOVE COSTRUZIONI VIA VIRGILIO T...,30,Appartamenti di nuova costruzione a Roma,ad,supervetrina,...,Lazio,Roma,"Battistini, Torrevecchia",Torresina,Roma,"{'id': 'IT', 'name': 'Italia', 'keyurl': 'Ital...",False,Progetto,"[Esposizione interna, Esposizione esterna, Acc...","14 in box/garage privato, 16 in parcheggio/gar..."
3,list-real-estate,sale,97472876,False,False,[{'caption': 'NUOVE COSTRUZIONI SENZA SPESE DI...,6,Attici e Appartamenti di nuova costruzione a Roma,ad,supervetrina,...,Lazio,Roma,"Acilia, Casal Bernocchi, Centro Giano, Dragona...",Dragona,Roma,"{'id': 'IT', 'name': 'Italia', 'keyurl': 'Ital...",False,Progetto,"[Esposizione interna, Esposizione esterna, Acc...",144 in parcheggio/garage comune
4,list-real-estate,sale,99215880,False,False,[{'caption': 'TORRESINA APPARTAMENTI NUOVA COS...,6,Appartamenti di nuova costruzione a Roma,ad,supervetrina,...,Lazio,Roma,"Battistini, Torrevecchia",Torresina,Roma,"{'id': 'IT', 'name': 'Italia', 'keyurl': 'Ital...",False,Progetto,"[Esposizione interna, Esposizione esterna, Acc...",30 in parcheggio/garage comune
5,list-real-estate,sale,97472182,False,False,[{'caption': 'Complesso immobiliare in Classe ...,4,Appartamenti di nuova costruzione a Roma,ad,supervetrina,...,Lazio,Roma,"Anagnina, Romanina, Tor Vergata",Romanina,Roma,"{'id': 'IT', 'name': 'Italia', 'keyurl': 'Ital...",False,Progetto,"[Esposizione interna, Esposizione esterna, Acc...",NaN
6,list-real-estate,sale,97471826,False,False,[{'caption': 'NUOVA COSTRUZIONE BOCCEA CASALOT...,9,Appartamenti e Attici di nuova costruzione a Roma,ad,supervetrina,...,Lazio,Roma,"Casalotti, Casal Selce, Maglianella",Casalotti,Roma,"{'id': 'IT', 'name': 'Italia', 'keyurl': 'Ital...",False,Progetto,"[Esposizione interna, Esposizione esterna, Acc...",77 in box/garage privato
7,list-real-estate,sale,102356094,False,False,[{'caption': 'COLLINAURELIA . ABITARE NEL VERD...,9,Appartamenti e Attici di nuova costruzione a Roma,ad,supervetrina,...,Lazio,Roma,"Casetta Mattei, Pisana, Bravetta",Pisana - Bravetta,Roma,"{'id': 'IT', 'name': 'Italia', 'keyurl': 'Ital...",False,Progetto,"[Esposizione interna, Esposizione esterna, Acc...",15 in parcheggio/garage comune
8,list-real-estate,sale,97492918,False,False,[{'caption': 'RESIDENZA CARTESIO - VIALE LEONI...,17,Appartamenti di nuova costruzione a Roma,ad,supervetrina,...,Lazio,Roma,"Castel di Leva, Vallerano",Castel di Leva,Roma,"{'id': 'IT', 'name': 'Italia', 'keyurl': 'Ital...",False,Progetto,"[Esposizione interna, Esposizione esterna, Acc...",NaN
9,list-real-estate,sale,100761287,False,False,"[{'category': {'id': 27, 'name': 'Nuove costru...",3,Appartamenti di nuova costruzione a Roma,ad,supervetrina,...,Lazio,Roma,"Magliana, Trullo, Parco de' Medici",Magliana,Roma,"{'id': 'IT', 'name': 'Italia', 'keyurl': 'Ital...",False,Progetto,"[Esposizione interna, Esposizione esterna, Acc...",34 in parcheggio/garage comune
